In [13]:
import sys
sys.path.insert(0, "..")

import scikitplot as skplt 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
import swifter
import matplotlib.pyplot as plt
plt.style.reload_library()
plt.style.use(['science'])

In [14]:
!ls -lah analysis/not_uploaded/IA/ | grep class

-rw-r--r-- 1 janpf janpf 712M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt
-rw-r--r-- 1 janpf janpf 836M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-three.change_class.epoch-1.pth.txt
-rw-r--r-- 1 janpf janpf 771M Nov  6 21:07 .scratch.ckpts.IA.pexels.scores-three.change_class.epoch-16.pth.txt


In [15]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt")

for col in df.columns:
    try:
        df[col] = df[col].swifter.apply(lambda row: eval(row)[0])
    except:
        pass

print(df.columns)
df

Pandas Apply: 100%|██████████| 2610000/2610000 [00:26<00:00, 97596.51it/s]
Index(['img', 'distortion', 'level', 'score', 'styles_contrast_class',
       'styles_brightness_class', 'styles_saturation_class',
       'styles_shadows_class', 'styles_highlights_class',
       'styles_exposure_class', 'styles_vibrance_class',
       'styles_temperature_class', 'styles_tint_class',
       'technical_jpeg_compression_class', 'technical_defocus_blur_class',
       'technical_motion_blur_class', 'technical_pixelate_class',
       'technical_gaussian_noise_class', 'technical_impulse_noise_class',
       'composition_rotate_class', 'composition_hcrop_class',
       'composition_vcrop_class', 'composition_leftcornerscrop_class',
       'composition_rightcornerscrop_class', 'composition_ratio_class'],
      dtype='object')


,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_motion_blur_class,technical_pixelate_class,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class
0,5/3/c/abbey-glass-religion-architecture-106158...,original,0.0,0.830078,3.623047,5.226562,7.175781,6.683594,7.171875,2.625000,...,0.398193,0.854980,1.997070,0.702148,2.726562,2.298828,2.359375,2.263672,2.322266,0.961914
1,9/4/4/abraham-lincoln-lincoln-memorial-washing...,original,0.0,0.570312,7.121094,9.171875,10.195312,8.843750,10.031250,9.453125,...,-0.735840,2.380859,0.815918,-0.728516,4.417969,3.250000,3.578125,3.154297,3.255859,3.960938
2,a/5/9/abstract-background-backgrounds-botany-8...,original,0.0,0.374756,7.789062,7.332031,7.851562,8.062500,7.734375,7.847656,...,0.638672,3.476562,-3.263672,-6.703125,4.800781,2.710938,2.888672,2.816406,2.835938,1.827148
3,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,original,0.0,0.553711,6.363281,7.136719,8.328125,8.984375,8.359375,7.527344,...,1.218750,-0.740234,0.238525,1.049805,4.042969,2.613281,2.833984,2.494141,2.542969,3.861328
4,a/0/7/adult-cute-face-female-41522.jpeg,original,0.0,0.407471,11.007812,13.500000,12.304688,13.398438,12.343750,10.914062,...,2.169922,4.164062,0.513184,-4.781250,4.218750,5.000000,5.097656,5.125000,5.171875,3.882812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,0.209473,1.983398,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,1.300781,2.632812,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,1.437500,4.968750,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,1.941406,3.630859,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000


In [16]:
df["intensity"] = [float("nan")]*len(df)
df.loc[df["level"] == 0, "intensity"] = 0
for t in ["styles", "technical", "composition"]:
    for dist in mapping[t].keys():
        for pol in mapping[t][dist].keys():
            if pol == "neg":
                for lvl, change in enumerate(reversed(mapping[t][dist][pol])):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = -(lvl+1)
            if pol == "pos":
                for lvl, change in enumerate(mapping[t][dist][pol]):
                    df.loc[(df["distortion"] == change.split(";")[0])&(df["level"] == float(change.split(";")[1])), "intensity"] = lvl+1
df["intensity"] = df["intensity"].astype(int)
df

,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_pixelate_class,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class,intensity
0,5/3/c/abbey-glass-religion-architecture-106158...,original,0.0,0.830078,3.623047,5.226562,7.175781,6.683594,7.171875,2.625000,...,0.854980,1.997070,0.702148,2.726562,2.298828,2.359375,2.263672,2.322266,0.961914,0
1,9/4/4/abraham-lincoln-lincoln-memorial-washing...,original,0.0,0.570312,7.121094,9.171875,10.195312,8.843750,10.031250,9.453125,...,2.380859,0.815918,-0.728516,4.417969,3.250000,3.578125,3.154297,3.255859,3.960938,0
2,a/5/9/abstract-background-backgrounds-botany-8...,original,0.0,0.374756,7.789062,7.332031,7.851562,8.062500,7.734375,7.847656,...,3.476562,-3.263672,-6.703125,4.800781,2.710938,2.888672,2.816406,2.835938,1.827148,0
3,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,original,0.0,0.553711,6.363281,7.136719,8.328125,8.984375,8.359375,7.527344,...,-0.740234,0.238525,1.049805,4.042969,2.613281,2.833984,2.494141,2.542969,3.861328,0
4,a/0/7/adult-cute-face-female-41522.jpeg,original,0.0,0.407471,11.007812,13.500000,12.304688,13.398438,12.343750,10.914062,...,4.164062,0.513184,-4.781250,4.218750,5.000000,5.097656,5.125000,5.171875,3.882812,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,1.983398,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344,5
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,2.632812,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383,5
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,4.968750,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703,5
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,3.630859,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000,5


In [17]:
df = df[(df["intensity"] == 5) | (df["intensity"] == -5)]
df

,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_pixelate_class,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class,intensity
5,5/3/c/abbey-glass-religion-architecture-106158...,contrast,-1.0,0.169434,9.343750,7.753906,7.734375,10.585938,8.203125,12.671875,...,-0.351318,-0.958984,-0.506836,3.707031,4.769531,4.828125,4.656250,4.902344,2.476562,-5
6,9/4/4/abraham-lincoln-lincoln-memorial-washing...,contrast,-1.0,0.385742,12.273438,13.914062,10.718750,11.968750,11.414062,10.843750,...,2.125000,-3.812500,-4.277344,5.566406,4.671875,4.839844,4.050781,5.183594,3.632812,-5
7,a/5/9/abstract-background-backgrounds-botany-8...,contrast,-1.0,0.699707,8.554688,10.437500,10.976562,10.687500,11.195312,9.054688,...,4.042969,-5.070312,-10.179688,4.273438,4.347656,4.902344,4.054688,5.445312,1.522461,-5
8,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,contrast,-1.0,0.150513,11.929688,11.062500,9.945312,11.640625,10.734375,14.539062,...,1.540039,-3.232422,-3.667969,3.486328,5.218750,5.863281,5.679688,5.691406,3.179688,-5
9,a/0/7/adult-cute-face-female-41522.jpeg,contrast,-1.0,0.203125,13.203125,15.710938,12.703125,14.187500,12.484375,14.195312,...,4.492188,-4.527344,-8.906250,4.968750,5.382812,5.710938,5.054688,5.785156,5.007812,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,1.983398,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344,5
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,2.632812,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383,5
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,4.968750,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703,5
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,3.630859,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000,5


In [18]:
dist_type = "composition"

df = df[df["distortion"].isin(mapping[dist_type].keys())]

In [19]:
def find_class(row:pd.Series, dist_type:str):
    relevant_cols = [c for c, _ in row.iteritems() if dist_type in c and "class" in c]
    m = -1
    m_c = ""
    for c in relevant_cols:
        if row[c] > m:
            m = row[c]
            m_c = c
    return m_c.split("_")[1]

In [20]:
df

,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_pixelate_class,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class,intensity
560,5/3/c/abbey-glass-religion-architecture-106158...,rotate,-10.0,0.923828,3.683594,7.519531,11.390625,9.710938,11.695312,3.992188,...,-0.222900,2.750000,1.804688,4.558594,3.708984,3.673828,3.869141,3.554688,-0.104614,-5
561,9/4/4/abraham-lincoln-lincoln-memorial-washing...,rotate,-10.0,0.591309,6.230469,8.718750,9.539062,7.765625,9.750000,8.804688,...,1.744141,-1.211914,-2.810547,4.179688,2.554688,2.914062,2.751953,2.474609,4.421875,-5
562,a/5/9/abstract-background-backgrounds-botany-8...,rotate,-10.0,0.399658,6.703125,6.167969,7.886719,7.019531,7.675781,6.539062,...,2.462891,-3.308594,-6.789062,3.988281,2.503906,2.794922,2.806641,2.630859,1.720703,-5
563,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,rotate,-10.0,0.661133,5.921875,7.031250,8.289062,8.570312,8.546875,6.359375,...,0.826172,-1.672852,0.121582,4.425781,2.339844,2.554688,1.765625,2.822266,3.701172,-5
564,a/0/7/adult-cute-face-female-41522.jpeg,rotate,-10.0,0.665039,9.937500,12.203125,11.773438,12.054688,11.695312,9.703125,...,2.597656,0.497070,-3.552734,4.222656,4.453125,4.500000,4.281250,4.628906,3.179688,-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,1.983398,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344,5
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,2.632812,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383,5
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,4.968750,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703,5
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,3.630859,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000,5


In [21]:
df[f"{dist_type}_class"] = df.apply(lambda row: find_class(row, dist_type), axis=1)
df

,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class,intensity,composition_class
560,5/3/c/abbey-glass-religion-architecture-106158...,rotate,-10.0,0.923828,3.683594,7.519531,11.390625,9.710938,11.695312,3.992188,...,2.750000,1.804688,4.558594,3.708984,3.673828,3.869141,3.554688,-0.104614,-5,rotate
561,9/4/4/abraham-lincoln-lincoln-memorial-washing...,rotate,-10.0,0.591309,6.230469,8.718750,9.539062,7.765625,9.750000,8.804688,...,-1.211914,-2.810547,4.179688,2.554688,2.914062,2.751953,2.474609,4.421875,-5,ratio
562,a/5/9/abstract-background-backgrounds-botany-8...,rotate,-10.0,0.399658,6.703125,6.167969,7.886719,7.019531,7.675781,6.539062,...,-3.308594,-6.789062,3.988281,2.503906,2.794922,2.806641,2.630859,1.720703,-5,rotate
563,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,rotate,-10.0,0.661133,5.921875,7.031250,8.289062,8.570312,8.546875,6.359375,...,-1.672852,0.121582,4.425781,2.339844,2.554688,1.765625,2.822266,3.701172,-5,rotate
564,a/0/7/adult-cute-face-female-41522.jpeg,rotate,-10.0,0.665039,9.937500,12.203125,11.773438,12.054688,11.695312,9.703125,...,0.497070,-3.552734,4.222656,4.453125,4.500000,4.281250,4.628906,3.179688,-5,rightcornerscrop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344,5,vcrop
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383,5,rightcornerscrop
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703,5,rotate
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000,5,rotate


In [22]:
df = df[["distortion", f"{dist_type}_class", "intensity"]]
df

,distortion,composition_class,intensity
560,rotate,rotate,-5
561,rotate,ratio,-5
562,rotate,rotate,-5
563,rotate,rotate,-5
564,rotate,rightcornerscrop,-5
...,...,...,...
2609995,ratio,vcrop,5
2609996,ratio,rightcornerscrop,5
2609997,ratio,rotate,5
2609998,ratio,rotate,5


In [23]:
if dist_type == "technical":
    df["distortion"] = df["distortion"].swifter.apply(lambda row: row.split("_")[0])

In [24]:
accuracy_score(df["distortion"], df[f"{dist_type}_class"])

0.2575388888888889